In [1]:
import os
import sys
import inspect
from dotenv import load_dotenv

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# to import backend module
currentdir = os.path.dirname(os.path.abspath(globals()['_dh'][0]))
if currentdir not in sys.path:
    sys.path.insert(0, currentdir) 

In [4]:
# env variables
load_dotenv('../.env.local')

True

In [5]:
from backend.ai import *

In [14]:
what_next("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: How many people live in canada as of 2023?
Thought:
-- langchain-mod --
{'prompt_tokens': 152, 'completion_tokens': 28, 'total_tokens': 180}
-- langchain-mod --

> Finished chain.
I need to find a reliable source for current population data for Canada.
Action: Search
Action Input: "Canada popula

As of January 1, 2023, Canada's population was estimated to be 39,566,248.
--
AiResult: used 715 token, cost $0.0014, took 12.0 seconds